In [34]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
device_nb = 1
os.environ["CUDA_VISIBLE_DEVICES"]=str(device_nb)

In [35]:
import pandas as pd
import math
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import scipy.stats
import random

In [36]:
########################################################
#raw data
########################################################



#subject_id = '38A'
blood_analyte = 'CGM'

xl = pd.ExcelFile('CGM_insulin_TG_data.xlsx')
Meal_dfs = xl.parse(blood_analyte)
sub_meal_idx = {}

sub_LBM = {'38A': 39.47,
           '38B': 62.78,
           '38C': 53.5,
           '38D': 37.47,
           '38E': 45.52,
           '38F': 43.38,
           '38H': 47.39,}

X=[]
Y1=[]
Y2=[]
Y3=[]

for index, row in Meal_dfs.iterrows():
    
    #ignore missing meals
    if not math.isnan(row.tolist()[6]):

        #get each subject's meal index
        for subject_id in ['38A', '38B', '38C', '38D', '38E', '38F', '38H']:
            if row['Patient_ID'] == subject_id:
                if subject_id in sub_meal_idx:
                    sub_meal_idx[subject_id] += [index]
                else:
                    sub_meal_idx[subject_id] = [index]
        
        
        
        
    X += [np.float32(one_cgm) for one_cgm in row.tolist()[2:34]]

    #carbs
    if row['Meal_ID'][4]== '1':
        Y1 += [np.float32(52.25/sub_LBM[row[0]])]
        #Y1 += [np.float32(1)]
    elif row['Meal_ID'][4]== '2':
        Y1 += [np.float32(94.75/sub_LBM[row[0]])]
        #Y1 += [np.float32(2)]
    elif row['Meal_ID'][4]== '3':
        Y1 += [np.float32(179.75/sub_LBM[row[0]])]
        #Y1 += [np.float32(3)]

    #protein
    if row['Meal_ID'][1]== '1':
        Y2 += [np.float32(15/sub_LBM[row[0]])]
        #Y2 += [np.float32(1)]
    elif row['Meal_ID'][1]== '2':
        Y2 += [np.float32(30/sub_LBM[row[0]])]
        #Y2 += [np.float32(2)]
    elif row['Meal_ID'][1]== '3':
        Y2 += [np.float32(60/sub_LBM[row[0]])]
        #Y2 += [np.float32(3)]

    #fat
    if row['Meal_ID'][7]== '1':
        Y3 += [np.float32(13/sub_LBM[row[0]])]    
        #Y3 += [np.float32(1)]  
    elif row['Meal_ID'][7]== '2':
        Y3 += [np.float32(26/sub_LBM[row[0]])]   
        #Y3 += [np.float32(2)]  
    elif row['Meal_ID'][7]== '3':
        Y3 += [np.float32(52/sub_LBM[row[0]])]  
        #Y3 += [np.float32(3)]



X_value = np.array(X).reshape(63,32)
scaler = StandardScaler()
X_value = scaler.fit_transform(X_value)
Y1_value = np.array(Y1).reshape(-1,1)
Y2_value = np.array(Y2).reshape(-1,1)
Y3_value = np.array(Y3).reshape(-1,1)

#Y1 = (Y1 - Y1.min())/(Y1.max()-Y1.min())
#Y2 = (Y2 - Y2.min())/(Y2.max()-Y2.min())
#Y3 = (Y3 - Y3.min())/(Y3.max()-Y3.min())

In [37]:
sub_meal_idx

{'38A': [0, 1, 2, 3, 4, 5, 6, 8],
 '38B': [10, 11, 12, 14, 15, 16, 17],
 '38C': [18, 20, 21, 22, 23, 24, 25, 26],
 '38D': [27, 28, 29, 30, 31, 32, 33, 34, 35],
 '38E': [36, 37, 38, 39, 40, 41, 42, 43, 44],
 '38F': [45, 46, 47, 48, 49, 50, 51, 52, 53],
 '38H': [54, 55, 56, 58, 59, 61, 62]}

In [38]:
########################################################
# 17 Gaussian AUC features (deprecated)
########################################################
"""
#A: miss 8
#B: miss 1, 5
#C: miss 2
#H: miss 4, 7
#the others: meal 1 2 3 4 5 6 7 8 9 

X_Gau = []


sub_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'H']
      
for sub_l in sub_letters:
    
    if sub_l == 'A':
        miss_meal = [8]
    elif sub_l == 'B':
        miss_meal = [1, 5]
    elif sub_l == 'C':
        miss_meal = [2]
    elif sub_l == 'H':
        miss_meal = [4, 7]
    else:
        miss_meal = []
    
    with open('./Tianlong/Gauss Features 2018 1121/%s.txt'%sub_l, 'r') as f0:
        
        # each line is a meal
        for line_num in range(1,10):
            
            one_meal  = []
            
            if line_num in miss_meal:
                
                X_Gau.append(np.array([np.nan for nan_num in range(17)]))
                
            else:
                line = f0.readline()

                for number in line.split(' '):

                    one_meal += [float(number)]

                one_meal = np.array(one_meal)
                X_Gau += [one_meal]
                
X_Gau = np.array(X_Gau)
"""

"\n#A: miss 8\n#B: miss 1, 5\n#C: miss 2\n#H: miss 4, 7\n#the others: meal 1 2 3 4 5 6 7 8 9 \n\nX_Gau = []\n\n\nsub_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'H']\n      \nfor sub_l in sub_letters:\n    \n    if sub_l == 'A':\n        miss_meal = [8]\n    elif sub_l == 'B':\n        miss_meal = [1, 5]\n    elif sub_l == 'C':\n        miss_meal = [2]\n    elif sub_l == 'H':\n        miss_meal = [4, 7]\n    else:\n        miss_meal = []\n    \n    with open('./Tianlong/Gauss Features 2018 1121/%s.txt'%sub_l, 'r') as f0:\n        \n        # each line is a meal\n        for line_num in range(1,10):\n            \n            one_meal  = []\n            \n            if line_num in miss_meal:\n                \n                X_Gau.append(np.array([np.nan for nan_num in range(17)]))\n                \n            else:\n                line = f0.readline()\n\n                for number in line.split(' '):\n\n                    one_meal += [float(number)]\n\n                one_meal = np.

In [39]:
########################################################
# 17 Gaussian AUC features
########################################################


data=pd.DataFrame(pd.read_excel('CGM_insulin_TG_data.xlsx'))


def process_data(data):
    sample_am = data.shape[0]
    protein = data.iloc[:, 1].values.reshape(-1, 1)
    cho = data.iloc[:, 2].values.reshape(-1, 1)
    fat = data.iloc[:, 3].values.reshape(-1, 1)
    # pudding_A = data_38A.iloc[:,4].values.reshape(-1,1)
    # water = data_38A.iloc[:,5].values.reshape(-1,1)
    total_e = data.iloc[:, 6].values.reshape(-1, 1)
    glucose = data.iloc[:, 2:35].values.reshape(sample_am, -1)
    #glucose_last_column = glucose[:, -1]
    #glucose_r = np.c_[glucose, glucose_last_column]
    return protein, cho, fat, total_e, glucose

def process_glucose(glucose):
    a = np.zeros((1, 5))
    for row in glucose:
        # print(glucose_gaussian_feature_4(row).shape())
        # print(glucose_gaussian_feature_8(row).shape())
        # print(glucose_gaussian_feature_16(row).shape())
        glucose_gaussian_feature = np.hstack([glucose_gaussian_feature_8(row)])
        a = np.vstack([a, glucose_gaussian_feature])
    # print(a.shape)
    a = np.delete(a, 0, axis=0)
    # print(a.shape)
    return a

def normal_function(mu, sigma, x):
    normal_function_value = (1 / (sigma * np.sqrt(2 * np.pi))) * \
        np.exp(-((x - mu) ** 2) / (2 * (sigma ** 2)))
    return normal_function_value

def glucose_gaussian_feature_4(glucose_row):
    gaussian_feature = []
    normal_g_1 = normal_function(
        0, 4/1.96, np.asarray([i for i in range(0, 5)])) * glucose_row[0:5]
    normal_g_2 = normal_function(
        4, 4/1.96, np.asarray([i for i in range(0, 9)])) * glucose_row[0:9]
    normal_g_3 = normal_function(
        8, 4/1.96, np.asarray([i for i in range(4, 13)])) * glucose_row[4:13]
    normal_g_4 = normal_function(
        12, 4/1.96, np.asarray([i for i in range(8, 17)])) * glucose_row[8:17]
    normal_g_5 = normal_function(
        16, 4/1.96, np.asarray([i for i in range(12, 21)])) * glucose_row[12:21]
    normal_g_6 = normal_function(
        20, 4/1.96, np.asarray([i for i in range(16, 25)])) * glucose_row[16:25]
    normal_g_7 = normal_function(
        24, 4/1.96, np.asarray([i for i in range(20, 29)])) * glucose_row[20:29]
    normal_g_8 = normal_function(
        28, 4/1.96, np.asarray([i for i in range(24, 33)])) * glucose_row[24:33]
    normal_g_9 = normal_function(
        32, 4/1.96, np.asarray([i for i in range(28, 33)])) * glucose_row[28:33]
    for i in range(1, 10):
        gaussian_feature.append(np.trapz(locals()["normal_g_" + str(i)]))
    return gaussian_feature

def glucose_gaussian_feature_8(glucose_row):
    gaussian_feature = []
    normal_g_1 = normal_function(
        0, 8/1.96, np.asarray([i for i in range(0, 9)])) * glucose_row[0:9]
    normal_g_2 = normal_function(
        8, 8/1.96, np.asarray([i for i in range(0, 17)])) * glucose_row[0:17]
    normal_g_3 = normal_function(
        16, 8/1.96, np.asarray([i for i in range(8, 25)])) * glucose_row[8:25]
    normal_g_4 = normal_function(
        24, 8/1.96, np.asarray([i for i in range(16, 33)])) * glucose_row[16:33]
    normal_g_5 = normal_function(
        32, 8/1.96, np.asarray([i for i in range(24, 33)])) * glucose_row[24:33]
    for i in range(1, 6):
        gaussian_feature.append(np.trapz(locals()["normal_g_" + str(i)]))
    return gaussian_feature

def glucose_gaussian_feature_16(glucose_row):
    gaussian_feature = []
    normal_g_1 = normal_function(
        0, 16/1.96, np.asarray([i for i in range(0, 17)])) * glucose_row[0:17]
    normal_g_2 = normal_function(
        16, 16/1.96, np.asarray([i for i in range(0, 33)])) * glucose_row[0:33]
    normal_g_3 = normal_function(
        32, 16/1.96, np.asarray([i for i in range(16, 33)])) * glucose_row[16:33]
    for i in range(1, 4):
        gaussian_feature.append(np.trapz(locals()["normal_g_" + str(i)]))
    return gaussian_feature


p,c,f,e,r=process_data(data)
newfeature=process_glucose(r)

X_Gau = newfeature

In [40]:
normal_function(0,8/1.96,8)

0.014318031361695611

In [41]:
######################################
#training + parameter search
######################################



import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy as np

#Y1=carbs     Y2=protein      Y3=fat


#options: X_value     X_Gau
Input_X = X_Gau

#loss => all default huber

#choose between L1SO vs. L1MO
scenario = 'L1SO'

L1MO_test_subs = ['38A', '38B', '38C', '38D', '38E', '38F', '38H']

neuron_shared_ls = [2, 3, 4, 5]

learning_rate_1_ls = [0.0005, 0.001, 0.005]

learning_rate_2_ls = [0.001, 0.005]

learning_rate_3_ls = [0.001, 0.005]





N_epochs = 111


#____________________________________________
#if not for grid search, can be commented out

for neurons_shared in neuron_shared_ls:
    for learning_rate_1 in learning_rate_1_ls:
        for learning_rate_2 in learning_rate_2_ls:
            for learning_rate_3 in learning_rate_3_ls:

                '''
                neurons_shared = 3
                learning_rate_1 = 0.001
                learning_rate_2 = 0.005
                learning_rate_3 = 0.005
                '''


                # ======================
                # Define the Graph
                # ======================

                # Define the Placeholders
                X = tf.placeholder("float", [None, Input_X.shape[1]], name="X")
                Y1 = tf.placeholder("float", [None, 1], name="Y1")
                Y2 = tf.placeholder("float", [None, 1], name="Y2")
                Y3 = tf.placeholder("float", [None, 1], name="Y3")

                # Define the weights for the layers
                initial_shared_layer_weights = np.random.rand(Input_X.shape[1],neurons_shared)
                initial_Y1_layer_weights = np.random.rand(neurons_shared,1)
                initial_Y2_layer_weights = np.random.rand(neurons_shared,1)
                initial_Y3_layer_weights = np.random.rand(neurons_shared,1)

                shared_layer_weights = tf.Variable(initial_shared_layer_weights, name="share_W", dtype="float32")
                Y1_layer_weights = tf.Variable(initial_Y1_layer_weights, name="share_Y1", dtype="float32")
                Y2_layer_weights = tf.Variable(initial_Y2_layer_weights, name="share_Y2", dtype="float32")
                Y3_layer_weights = tf.Variable(initial_Y3_layer_weights, name="share_Y3", dtype="float32")



                #Define the biases for the layers
                initial_shared_layer_bias = np.random.rand(neurons_shared,)
                initial_Y1_layer_bias = np.random.rand(1)
                initial_Y2_layer_bias = np.random.rand(1)
                initial_Y3_layer_bias = np.random.rand(1)

                shared_layer_bias = tf.Variable(initial_shared_layer_bias, name="share_B", dtype="float32")
                Y1_layer_bias = tf.Variable(initial_Y1_layer_bias, name="share_Y1_B", dtype="float32")
                Y2_layer_bias = tf.Variable(initial_Y2_layer_bias, name="share_Y2_B", dtype="float32")
                Y3_layer_bias = tf.Variable(initial_Y3_layer_bias, name="share_Y3_B", dtype="float32")



                # Construct the Layers with RELU Activations
                shared_layer = tf.nn.relu(tf.add(tf.matmul(X,shared_layer_weights), shared_layer_bias))
                #Y1_layer = tf.nn.relu(tf.add(tf.matmul(shared_layer,Y1_layer_weights), Y1_layer_bias))
                #Y2_layer = tf.nn.relu(tf.add(tf.matmul(shared_layer,Y2_layer_weights), Y2_layer_bias))
                #Y3_layer = tf.nn.relu(tf.add(tf.matmul(shared_layer,Y3_layer_weights), Y3_layer_bias))
                Y1_layer = tf.add(tf.matmul(shared_layer,Y1_layer_weights), Y1_layer_bias)
                Y2_layer = tf.add(tf.matmul(shared_layer,Y2_layer_weights), Y2_layer_bias)
                Y3_layer = tf.add(tf.matmul(shared_layer,Y3_layer_weights), Y3_layer_bias)




                # Calculate Loss
                #Y1_Loss = tf.nn.l2_loss(Y1-Y1_layer)
                #Y1_Loss = tf.norm(Y1-Y1_layer, ord= 1)
                #Y1_Loss = tf.losses.mean_squared_error(Y1, Y1_layer)
                Y1_Loss = tf.losses.huber_loss(Y1, Y1_layer)
                #Y1_Loss = tf.keras.losses.logcosh(Y1, Y1_layer)

                #Y2_Loss = tf.nn.l2_loss(Y2-Y2_layer)
                #Y2_Loss = tf.norm(Y2-Y2_layer, ord= 1)
                #Y2_Loss = tf.losses.mean_squared_error(Y2, Y2_layer)
                Y2_Loss = tf.losses.huber_loss(Y2, Y2_layer)

                #Y3_Loss = tf.nn.l2_loss(Y3-Y3_layer)
                #Y3_Loss = tf.norm(Y3-Y3_layer, ord= 1)
                #Y3_Loss = tf.losses.mean_squared_error(Y3, Y3_layer)
                Y3_Loss = tf.losses.huber_loss(Y3, Y3_layer)

                Joint_Loss = Y1_Loss + Y2_Loss + Y3_Loss

                # optimisers
                Optimiser = tf.train.AdamOptimizer().minimize(Joint_Loss)
                Y1_op = tf.train.AdamOptimizer(learning_rate=learning_rate_1, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(Y1_Loss)
                Y2_op = tf.train.AdamOptimizer(learning_rate=learning_rate_2, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(Y2_Loss)
                Y3_op = tf.train.AdamOptimizer(learning_rate=learning_rate_3, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(Y3_Loss)










                #__________________________________________________________________________________________________________
                # Joint Training
                # Calculation (Session) Code
                # ==========================

                # open the session
                session = tf.InteractiveSession()




                ############### L1SO ####################
                if scenario == 'L1SO':

                    sub_to_correlation = {}


                    for test_sub in ['38A', '38B', '38C', '38D', '38E', '38F', '38H']: 


                        if test_sub not in sub_to_correlation:
                            sub_to_correlation[test_sub] = {}


                        Y1_loss_list = []
                        Y2_loss_list = []
                        Y3_loss_list = []
                        Joint_loss_list = []


                        session.run(tf.global_variables_initializer())


                        #######  training phase  ###########
                        for i in range(1, N_epochs):


                            train_meal_idx = []
                            for one_sub in sub_meal_idx:
                                if one_sub != test_sub:
                                    train_meal_idx += sub_meal_idx[one_sub]

                            random.shuffle(train_meal_idx)


                            #each meal will be one batch, and only update one out of three loss
                            for meal_idx in train_meal_idx:
                                rand_nb = np.random.rand()
                                if rand_nb < 0.25:
                                    _, Y1_loss = session.run([Y1_op, Y1_Loss],
                                                    {
                                                      X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                      Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                      Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                      Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                      })

                                    Y1_loss_list += [float(str(Y1_loss))]

                                elif 0.25 <= rand_nb < 0.5:
                                    _, Y2_loss = session.run([Y2_op, Y2_Loss],
                                                    {
                                                      X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                      Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                      Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                      Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                      })

                                    Y2_loss_list += [float(str(Y2_loss))]

                                elif 0.5 <= rand_nb < 0.75:
                                    _, Y3_loss = session.run([Y3_op, Y3_Loss],
                                                    {
                                                      X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                      Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                      Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                      Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                      })

                                    Y3_loss_list += [float(str(Y3_loss))]

                                else:
                                    _, Joint_loss = session.run([Optimiser, Joint_Loss],
                                                    {
                                                      X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                      Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                      Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                      Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                      })
                                    Joint_loss_list += [float(str(Joint_loss))]



                            if i % 2000 == 0:
                                print('y1 loss: ', Y1_loss, end=' |')
                                print('y2 loss: ', Y2_loss, end=' |')
                                print('y3 loss: ', Y3_loss, end=' |')
                                print('joint loss: ', Joint_loss)



                        #######  testing phase  ###########
                        all_pred = []
                        all_true = []

                        for test_meal in sub_meal_idx[test_sub]:
                            Y1_pred, Y2_pred, Y3_pred = session.run([Y1_layer,Y2_layer,Y3_layer], {
                                                  X: Input_X[test_meal].reshape(-1,Input_X.shape[1]),
                                                  Y1: Y1_value[test_meal].reshape(-1,1),
                                                  Y2: Y2_value[test_meal].reshape(-1,1),
                                                  Y3: Y3_value[test_meal].reshape(-1,1)
                                                  })
                            all_pred += [[Y1_pred[0][0], Y2_pred[0][0], Y3_pred[0][0]]]
                            all_true += [[Y1_value[test_meal][0], Y2_value[test_meal][0], Y3_value[test_meal][0]]]



                        #######  plotting phase  ###########
                        corr_cx = []
                        corr_cy = []
                        corr_px = []
                        corr_py = []
                        corr_fx = []
                        corr_fy = []


                        for meal_num in range(len(all_true)):
                            corr_cx.append(all_pred[meal_num][0])
                            corr_cy.append(all_true[meal_num][0])

                            corr_px.append(all_pred[meal_num][1])
                            corr_py.append(all_true[meal_num][1])

                            corr_fx.append(all_pred[meal_num][2])
                            corr_fy.append(all_true[meal_num][2])    

                        sub_to_correlation[test_sub] = {'c':scipy.stats.pearsonr(corr_cx,corr_cy),
                                                        'p':scipy.stats.pearsonr(corr_px,corr_py),
                                                        'f':scipy.stats.pearsonr(corr_fx,corr_fy)}

                        
                       
                        
                        '''
                        plt.subplot(411)
                        plt.plot(Y1_loss_list)
                        plt.subplot(412)
                        plt.plot(Y2_loss_list)
                        plt.subplot(413)
                        plt.plot(Y3_loss_list)
                        plt.subplot(414)
                        plt.plot(Joint_loss_list)
                        plt.show()
                        '''
                        print(test_sub[-1])
                        print(corr_cx)
                        print('truth_'+test_sub[-1]+'_CHO = '+corr_cx)
                        print(test_sub[-1]+'_CHO = '+corr_cy)
                        print('truth_'+test_sub[-1]+'_pro = '+corr_px)
                        print(test_sub[-1]+'_pro = '+corr_py)
                        print('truth_'+test_sub[-1]+'_fat = '+corr_fx)
                        print(test_sub[-1]+'_fat = '+corr_fy)
                        
                        
                    print(neurons_shared, learning_rate_1, learning_rate_2, learning_rate_3) 
                    print(sub_to_correlation)
                        

                    print(max(Y1_loss_list))
                    print(max(Y2_loss_list))  
                    print(max(Y3_loss_list))



                ################# L1MO ####################       
                elif scenario == 'L1MO':

                    sub_to_correlation = {}

                    #for test_sub in ['38A', '38B', '38C', '38D', '38E', '38F', '38H']: 
                    #for test_sub in ['38B']: 
                    for test_sub in L1MO_test_subs:

                        if test_sub not in sub_to_correlation:
                             sub_to_correlation[test_sub] = {}


                        Y1_loss_list = []
                        Y2_loss_list = []
                        Y3_loss_list = []
                        Joint_loss_list = []

                        session.run(tf.global_variables_initializer())

                        all_meal_pred = []
                        all_meal_true = []

                        for test_meal in sub_meal_idx[test_sub]:

                            #create training indexes
                            train_meal_idxes = []
                            for meal_index in sub_meal_idx[test_sub]:
                                if meal_index != test_meal:
                                    train_meal_idxes.append(meal_index) 

                            print(test_sub, test_meal, train_meal_idxes)
                            random.shuffle(train_meal_idxes)


                            #######  training phase  ###########
                            for i in range(1,N_epochs):
                                for meal_idx in train_meal_idxes:

                                    rand_nb = np.random.rand()
                                    if rand_nb < 0.333:
                                        _, Y1_loss = session.run([Y1_op, Y1_Loss],
                                                        {
                                                          X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                          Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                          Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                          Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                          })

                                        Y1_loss_list += [float(str(Y1_loss))]

                                    elif 0.333 <= rand_nb < 0.666:
                                        _, Y2_loss = session.run([Y2_op, Y2_Loss],
                                                        {
                                                          X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                          Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                          Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                          Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                          })

                                        Y2_loss_list += [float(str(Y2_loss))]

                                    elif 0.666 <= rand_nb < 0.999:
                                        _, Y3_loss = session.run([Y3_op, Y3_Loss],
                                                        {
                                                          X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                          Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                          Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                          Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                          })

                                        Y3_loss_list += [float(str(Y3_loss))]

                                    '''    
                                    else:
                                        _, Joint_loss = session.run([Optimiser, Joint_Loss],
                                                        {
                                                          X: Input_X[meal_idx, :].reshape(-1,Input_X.shape[1]),
                                                          Y1: Y1_value[meal_idx, :].reshape(-1,1),
                                                          Y2: Y2_value[meal_idx, :].reshape(-1,1),
                                                          Y3: Y3_value[meal_idx, :].reshape(-1,1)
                                                          })
                                        Joint_loss_list += [float(str(Joint_loss))]
                                    '''


                            if i % 50 == 0:
                                print('y1 loss: ', Y1_loss, end=' |')
                                print('y2 loss: ', Y2_loss, end=' |')
                                print('y3 loss: ', Y3_loss, end=' |')
                                print('joint loss: ', Joint_loss)



                            #######  testing phase  ###########
                            Y1_pred, Y2_pred, Y3_pred = session.run([Y1_layer,Y2_layer,Y3_layer], {
                                                  X: Input_X[test_meal].reshape(-1,Input_X.shape[1]),
                                                  Y1: Y1_value[test_meal].reshape(-1,1),
                                                  Y2: Y2_value[test_meal].reshape(-1,1),
                                                  Y3: Y3_value[test_meal].reshape(-1,1)
                                                  })

                            all_meal_pred += [[Y1_pred[0][0], Y2_pred[0][0], Y3_pred[0][0]]]
                            all_meal_true += [[Y1_value[test_meal][0], Y2_value[test_meal][0], Y3_value[test_meal][0]]]


                        #######  plotting phase  ###########
                        corr_cx = []
                        corr_cy = []
                        corr_px = []
                        corr_py = []
                        corr_fx = []
                        corr_fy = []


                        for meal_num in range(len(all_meal_true)):
                            corr_cx.append(all_meal_pred[meal_num][0])
                            corr_cy.append(all_meal_true[meal_num][0])

                            corr_px.append(all_meal_pred[meal_num][1])
                            corr_py.append(all_meal_true[meal_num][1])

                            corr_fx.append(all_meal_pred[meal_num][2])
                            corr_fy.append(all_meal_true[meal_num][2])    

                        sub_to_correlation[test_sub] = {'c':scipy.stats.pearsonr(corr_cx,corr_cy),
                                                        'p':scipy.stats.pearsonr(corr_px,corr_py),
                                                        'f':scipy.stats.pearsonr(corr_fx,corr_fy)}

                        
                        
                        '''
                        plt.subplot(411)
                        plt.plot(Y1_loss_list)
                        plt.subplot(412)
                        plt.plot(Y2_loss_list)
                        plt.subplot(413)
                        plt.plot(Y3_loss_list)
                        plt.subplot(414)
                        plt.plot(Joint_loss_list)
                        plt.show()
                        '''
                        
                    print(neurons_shared, learning_rate_1, learning_rate_2, learning_rate_3)    
                    print(sub_to_correlation)
                    
                    
                   


                    #print(max(Y1_loss_list))
                    #print(max(Y2_loss_list))  
                    #print(max(Y3_loss_list))


session.close()
from numba import cuda
cuda.select_device(0)
cuda.close()

/home/grads/g/guangzhou92/enter/envs/py36/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


A
[1.8088055, 1.9926975, 1.3764513, 1.3764513, 3.0154686, 1.3764513, 1.3853207, 1.6126454]


TypeError: can only concatenate str (not "list") to str

In [ ]:
sub_to_correlation

In [ ]:
type(sub_to_correlation['38A']['c'][1])

In [ ]:
train_meal_idxes

In [ ]:
test_sub

In [ ]:
sub_to_correlation

In [ ]:
print(session.run(Joint_Loss, {
                          X: Input_X[sub_meal_idx[test_sub][2]].reshape(-1,Input_X.shape[1]),
                          Y1: Y1_value[sub_meal_idx[test_sub][2]].reshape(-1,1),
                          Y2: Y2_value[sub_meal_idx[test_sub][2]].reshape(-1,1),
                          Y3: Y3_value[sub_meal_idx[test_sub][2]].reshape(-1,1)
                          }))

In [ ]:
all_pred = []
all_true = []


for test_meal in sub_meal_idx[test_sub]:
    Y1_pred, Y2_pred, Y3_pred = session.run([Y1_layer,Y2_layer,Y3_layer], {
                          X: Input_X[test_meal].reshape(-1,Input_X.shape[1]),
                          Y1: Y1_value[test_meal].reshape(-1,1),
                          Y2: Y2_value[test_meal].reshape(-1,1),
                          Y3: Y3_value[test_meal].reshape(-1,1)
                          })
    all_pred += [[Y1_pred[0][0], Y2_pred[0][0], Y3_pred[0][0]]]
    all_true += [[Y1_value[test_meal][0], Y2_value[test_meal][0], Y3_value[test_meal][0]]]
    print(Y1_pred, Y2_pred, Y3_pred)
    print(Y1_value[test_meal], Y2_value[test_meal], Y3_value[test_meal])

In [ ]:
all_pred

In [ ]:
all_true

In [ ]:
corr_cx = []
corr_cy = []
corr_px = []
corr_py = []
corr_fx = []
corr_fy = []


for meal_num in range(len(all_true)):
    corr_cx.append(all_pred[meal_num][0])
    corr_cy.append(all_true[meal_num][0])
    
    corr_px.append(all_pred[meal_num][1])
    corr_py.append(all_true[meal_num][1])
    
    corr_fx.append(all_pred[meal_num][2])
    corr_fy.append(all_true[meal_num][2])

In [ ]:
scipy.stats.pearsonr(corr_cx,corr_cy)

In [ ]:
scipy.stats.pearsonr(corr_px,corr_py)

In [ ]:
scipy.stats.pearsonr(corr_fx,corr_fy)